# Aula 1 - Modelo de Regressão

## 1.1 Apresentação

## 1.2 Carregando os dados

[JSON Viewer](http://jsonviewer.stack.hu)

[Pandas: read_json](https://pandas.pydata.org/docs/reference/api/pandas.read_json.html?highlight=read_json#pandas.read_json)


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dados = pd.read_json(
    path_or_buf= '/content/imoveis.json',
    orient='columns'
)

dados

### Explorar as informações

In [ ]:
dados.ident[0]

In [ ]:
dados.listing[0]

## 1.3 Tratamento de dados inicial

### Normalização

In [ ]:
dados_lista1 = pd.json_normalize(dados.ident)
dados_lista1.head()

In [ ]:
dados_lista2 = pd.json_normalize(dados.listing, sep='_')
dados_lista2.head()

### Junção dos dataframes

In [ ]:
dados_imoveis = pd.concat([dados_lista1, dados_lista2], axis=1)
dados_imoveis.head()

In [ ]:
dados_imoveis.shape

In [ ]:
for coluna in dados_imoveis.columns:
    print('========='*20)
    print(dados_imoveis[coluna].value_counts())

## 1.4 Limpeza

### Filtro

In [ ]:
filtro = (dados_imoveis['types_usage'] == 'Residencial') & (dados_imoveis['address_city'] ==  'Rio de Janeiro')

#aplicando o filtro
dados_imoveis = dados_imoveis[filtro]
dados_imoveis.head()

In [ ]:
dados_imoveis.shape

### Tipos de variáveis

In [ ]:
#atenção ao tipo das variáveis e o indice
dados_imoveis.info()

Verificando a saída do `.info`

> 69588 entries, 0 to 73614

 percebemos que temos que resentar o index, para que as infos fiquem ordenadas corretamente.


In [ ]:
dados_imoveis.reset_index(drop=True, inplace=True)

In [ ]:
#mostrar o RangeIndex
dados_imoveis.info(verbose=False)

Agora está correto.

In [ ]:
dados_imoveis = dados_imoveis.astype({
    'prices_price': 'float64',
    'prices_tax_iptu': 'float64',
    'prices_tax_condo': 'float64',
    'features_usableAreas': 'int64',
    'features_totalAreas': 'int64',
})

dados_imoveis.info()

**Uma empresa que atua em estados do Brasil contratou você para tratar os dados dos clientes dela. Quando você recebe o DataFrame chamado clientes_capital, percebe que ele contém os dados de vários clientes que moram em capitais do Brasil. No entanto, foi notado que alguns clientes não tinham os estados correspondentes às suas capitais, conforme mostra a tabela abaixo:**

In [ ]:
import pandas as pd
from numpy import nan

clientes_capital = pd.DataFrame({'Estados':['Ceará','Paraná','Piauí',nan,nan,'Roraima','São Paulo',nan,'Paraná','Piauí',nan,nan,'Minas Gerais',nan,'Piauí',nan,nan],
                           'Capitais':['Fortaleza','Curitiba','Teresina','Boa Vista','Curitiba','Boa Vista','São Paulo','Teresina','Curitiba','Teresina','São Paulo','Fortaleza','Belo Horizonte','Curitiba','Teresina','Belo Horizonte','Fortaleza'],
                           'id_clientes':[1830,3219,3149,3100,5754,648,3006,1139,2460,4748,2608,60,65,7457,3305,4397,3438]})

clientes_capital

Não é desejado remover as linhas em branco de clientes_capital, então como você poderia construir o código para preencher os dados dos estados? Indique o código que substitui os dados corretamente.

In [ ]:
filtro = ~clientes_capital['Estados'].isna()
dic_sem_duplicatas = clientes_capital[filtro].drop_duplicates(subset=['Capitais']).to_dict('records')
dic_sem_duplicatas

In [ ]:
dic_estado_capital = {selecao['Capitais']: selecao['Estados'] for selecao in dic_sem_duplicatas}
dic_estado_capital

In [ ]:
for capital, estado in dic_estado_capital.items():
  filtro = clientes_capital['Capitais'] == capital
  clientes_capital.loc[filtro, 'Estados'] = estado
clientes_capital

## 1.5 - Tratamento de dados final

### Lidando com dados nulos

#### Dados nulos zone

In [ ]:
dados_imoveis.address_zone.value_counts()

In [ ]:
dados_imoveis['address_zone'] = dados_imoveis['address_zone'].replace('', np.nan)

In [ ]:
#verificação
dados_imoveis.address_zone.isnull().sum()

In [ ]:
#relação zona com bairro
dados_imoveis.head()

Preencher a zona a partir da informação do bairro

In [ ]:
type(dados_imoveis[dados_imoveis['address_zone'].isnull()]['address_zone'][382])

In [ ]:
dados_imoveis.query("address_zone != address_zone")

In [ ]:
dados_imoveis.query("address_zone.isnull()", engine='python')

In [ ]:
dici = dados_imoveis[~dados_imoveis['address_zone'].isna()].drop_duplicates(subset=['address_neighborhood']).to_dict('records')
print(dici)

In [ ]:
#dicionario da bairro para zona
dic_zonas = {dic['address_neighborhood']: dic['address_zone'] for dic in dici}
print(dic_zonas)

In [ ]:
for bairro,zona in dic_zonas.items(): dados_imoveis.loc[dados_imoveis['address_neighborhood'] == bairro, 'address_zone'] = zona

In [ ]:
dados_imoveis['address_zone'].isnull().sum()

#### Dados nulos prices

In [ ]:
dados_imoveis.prices_tax_iptu.isnull().sum()

In [ ]:
dados_imoveis.prices_tax_condo.isnull().sum()

In [ ]:
dados_imoveis.prices_tax_iptu.fillna(0, inplace=True)
dados_imoveis.prices_tax_condo.fillna(0, inplace=True)

In [ ]:
print(f"Total de prices nulos: {dados_imoveis.prices_tax_iptu.isnull().sum()}")
print(f"Total de prices_tax_condo nulos: {dados_imoveis.prices_tax_condo.isnull().sum()}")

### Alterar o nome das colunas e seleção de variáveis

In [ ]:
dados_imoveis.drop(['customerID', 'source', 'types_usage', 'address_city',
                    'address_location_lon', 'address_location_lat','address_neighborhood' ], axis = 1, inplace=True)

In [ ]:
dicionario_colunas = {'types_unit': 'unit', 'address_zone': 'zone', 'prices_price': 'price',
                     'prices_tax_condo': 'tax_condo', 'prices_tax_iptu': 'tax_iptu', 'features_bedrooms': 'bedrooms',
                      'features_bathrooms': 'bathrooms', 'features_suites': 'suites', 'features_parkingSpaces': 'parkingSpaces',
                      'features_usableAreas': 'usableAreas', 'features_totalAreas':'totalAreas', 'features_floors':'floors',
                      'features_unitsOnTheFloor':'unitsOnTheFloor', 'features_unitFloor':'unitFloor'}

dados_imoveis = dados_imoveis.rename(dicionario_colunas, axis = 1)
dados_imoveis.head()

# Aula 2 - Regressão linear

## 2.1 - Correlação

*   Correlação: grau da relação entre variáveis. Pode ser positiva ou negativa essa relação.

*   Multicolinearidade: relação perfeita ou exata entre variáveis.



**Dica de leitura:** [Estatística prática para cientistas de dados: 50 conceitos essenciais](https://www.amazon.com.br/Estat%C3%ADstica-Pr%C3%A1tica-Para-Cientistas-Dados/dp/855080603X/ref=sr_1_21?__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=1JJ44C5LJXLR3&keywords=data+science&qid=1657892785&sprefix=data+science%2Caps%2C201&sr=8-21)
e [Estatística: O que é, para que serve e como funciona](https://www.amazon.com.br/Estat%C3%ADstica-para-serve-como-funciona/dp/8537815128/ref=pd_sbs_sccl_3_1/137-9319794-7502061?pd_rd_w=EQoo7&content-id=amzn1.sym.d27ac626-cb3c-4c30-830f-0fd405f2510f&pf_rd_p=d27ac626-cb3c-4c30-830f-0fd405f2510f&pf_rd_r=E896BYW6W206YVVR3SHA&pd_rd_wg=3O0Cg&pd_rd_r=3e3910b0-5f34-4a57-87b2-1d918494d614&pd_rd_i=8537815128&psc=1) [(Naked Statistics: Stripping the Dread from the Data - versão em inglês)](https://www.amazon.com.br/Naked-Statistics-Stripping-Dread-Data/dp/039334777X/ref=sr_1_2?keywords=naked+statistics&qid=1657893464&sprefix=naked+st%2Caps%2C267&sr=8-2&ufe=app_do%3Aamzn1.fos.6a09f7ec-d911-4889-ad70-de8dd83c8a74)

In [ ]:
correlacao = dados_imoveis.corr()
correlacao

In [ ]:
#bibliotecas para visualização
import matplotlib.pyplot as plt
import seaborn as sns

Para mais detalhes consultar [Documentação Seaborn](https://seaborn.pydata.org/generated/seaborn.heatmap.html)

In [ ]:
#visualização das correlações
cores = sns.color_palette('light:salmon', as_cmap=True)
mask = np.zeros_like(correlacao)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(13, 8))
    ax = sns.heatmap(correlacao, cmap=cores, mask=mask, square=True, fmt='.2f', annot=True)

Análise da distribuição da variável resposta.

In [ ]:
#antes (distrib. assimetrica a direita)
ax = sns.histplot(data=dados_imoveis, x='price', kde=True)
ax.figure.set_size_inches(20, 10)
ax.set_title('Histograma de preços')
ax.set_xlabel('Preço');

### Transformação das variáveis

Estamos realizando uma transformação logarítmica das variáveis para obtermos uma distribuição simétrica (mais próxima da normal), para que possamos usar o modelo de regressão linear. A transformação realizada é:


$$log1p = log(1+x)$$


Lembrar de realizar a transformação inversa para obter o valor original:


$$expm1 = exp(x) - 1$$




* Para mais detalhes consultar as documentações [`log1p`](https://numpy.org/doc/stable/reference/generated/numpy.log1p.html), [`expm1`](https://numpy.org/doc/stable/reference/generated/numpy.expm1.html#numpy.expm1) e [`FunctionTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html#sklearn.preprocessing.FunctionTransformer)



In [ ]:
from sklearn.preprocessing import FunctionTransformer

transformer = FunctionTransformer(np.log1p, validate=True)
dados_transformados = transformer.transform(dados_imoveis.select_dtypes(exclude=['object']))

In [ ]:
dados_transformados

In [ ]:
colunas_dados_tranformados = dados_imoveis.select_dtypes(exclude=['object']).columns

df_transformado = pd.concat([dados_imoveis.select_dtypes(include=['object']), pd.DataFrame(dados_transformados, columns=colunas_dados_tranformados)], axis=1)
df_transformado.head()

In [ ]:
#depois da transformação
correlacao_transformado = df_transformado.corr()
mask = np.zeros_like(correlacao_transformado)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(13, 8))
    ax = sns.heatmap(correlacao_transformado, cmap=cores, mask=mask, square=True, fmt='.2f', annot=True)
    ax.set_title('Correlação entre variáveis - Tranformação Log', fontsize=20)

In [ ]:
#depois (distrib. simétrica)
ax = sns.histplot(data=df_transformado, x='price', kde=True)
ax.figure.set_size_inches(20, 10)
ax.set_title('Histograma de preços')
ax.set_xlabel('Preço');

### Dummies

In [ ]:
variaveis_categoricas = df_transformado.select_dtypes(include=['object']).columns

In [ ]:
variaveis_categoricas

In [ ]:
df_transformado[variaveis_categoricas]

In [ ]:
df_dummies = pd.get_dummies(df_transformado[variaveis_categoricas])
df_dummies.head()

In [ ]:
dados_imoveis_dummies = pd.concat([df_transformado.drop(variaveis_categoricas, axis=1), df_dummies], axis=1)
dados_imoveis_dummies.head()

## 2.2 - Regressão Linear

* "A análise de regressão diz respeito ao estudo da dependência de uma variável, a variável dependente, em relação a uma ou mais variáveis, as variáveis explanatórias, visando estimar e/ou prever o valor médio (da população) da primeira em termos dos valores conhecidos ou fixados (em amostragens repetidas) das segundas."

* [Documentação Sklearn - LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression)

* Regressão Simples: a variável dependente (regressando ou `y`) se relaciona a uma única variável explanatória (regressor ou `X`).

* Regressão Múltipla: a variável dependente (regressando ou `y`) se relaciona a mais de uma única variável explanatória (regressor ou `Xi`).


[slides]

## 2.3 - Ajuste e Previsão

### Divisão dos dados

In [ ]:
#variáveis explanatórias (independentes)
X = dados_imoveis_dummies.drop('price', axis=1)

#variável dependente
y = dados_imoveis_dummies['price']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#divisão em conjunto de treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, random_state=42)

### Modelo

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
#Instanciando o modelo
lr = LinearRegression()

In [ ]:
#treino
lr.fit(X_treino, y_treino)

In [ ]:
#teste
previsao_lr = lr.predict(X_teste)

In [ ]:
#resultados da previsão
previsao_lr

In [ ]:
#transformação inversa
np.expm1(13.25768024)

## 2.4 - Métrica de desempenho

### Coeficiente de Determinação

* descobrir quão “bem” uma linha de regressão amostral é adequada aos dados
* r2 situa-se entre 0 e 1
* "r2 mede a proporção ou percentual da variação total de Y explicada pelo modelo de regressão."





In [ ]:
#importar a biblioteca
from sklearn.metrics import r2_score

In [ ]:
r2_lr = r2_score(y_teste, previsao_lr)
r2_lr

**Gráfico: 2 linhas (verdade e previsão - Teste)**

In [ ]:
tam_test = len(y_teste)
tam_test

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data for plotting
t = np.arange(0, tam_test, 1)

fig, ax = plt.subplots()
ax.plot(t, np.expm1(y_teste), 'b')

fig, ax = plt.subplots()
ax.plot(t, np.expm1(previsao_lr), 'r')

In [ ]:
# Data for plotting
en=20
t = np.arange(0, en, 1)

fig, ax = plt.subplots()
ax.plot(t, np.expm1(y_teste[:en]), 'b')
ax.plot(t, np.expm1(previsao_lr[:en]), 'r')

### Extra

* [Documentação Sklearn - Métricas para Regressão](https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics)

* Observação: Estas métricas dependem da escala dos dados!!!

* MSE (Mean Squared Error ou Erro Quadrático Médio): média da diferença **elevada ao quadrado** entre o valor real e o previsto. *(penalidade sobre o erro)*

* MAE (Mean Absolute Error ou Erro Absoluto Médio): média da diferença **absoluta** (módulo) entre o valor real e o previsto.


In [ ]:
from sklearn.metrics import mean_squared_error as mse

mse_lr = mse(y_teste, previsao_lr)
mse_lr

In [ ]:
from sklearn.metrics import mean_absolute_error as mae

mae_lr = mae(y_teste, previsao_lr)
mae_lr

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error as mape

mape_lr = mape(y_teste, previsao_lr)
mape_lr

# Aula 3 - Árvore de Decisão para Regressão






## 3.1 - Árvore de Decisão Classificação versus Regressão

[slides]

[Biblioteca Sklearn - Desicion Tree](https://scikit-learn.org/stable/modules/tree.html#regression)

**Dica de leitura**: [Capítulo 6 - Mãos à obra: aprendizado de máquina com Scikit-Learn, Keras & TensorFlow: Conceitos, ferramentas e técnicas para a construção de sistemas inteligentes](https://www.amazon.com.br/M%C3%A3os-obra-aprendizado-Scikit-Learn-inteligentes/dp/8550815489/ref=sr_1_2?keywords=machine+learning&qid=1658181249&sprefix=machine%2Caps%2C469&sr=8-2&ufe=app_do%3Aamzn1.fos.6d798eae-cadf-45de-946a-f477d47705b9), [Para saber mais: como funciona a árvore de decisão? - Classificação](https://cursos.alura.com.br/course/machine-learning-classificacao-tras-panos/task/107966)



## 3.2 - Implementando o Modelo


In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
#Instanciando o modelo
dtr = DecisionTreeRegressor(random_state=42, max_depth=15)

In [ ]:
#Treino
dtr.fit(X_treino, y_treino)

In [ ]:
#Teste
previsao_dtr = dtr.predict(X_teste)

In [ ]:
#Resultados da previsão
previsao_dtr

**Gráfico de erro de predição**: Um gráfico de erros de predição mostra os alvos reais em relação aos valores previstos. Em um modelo perfeito, esses pontos estariam alinhados em 45 grau.


Para saber mais sobre outras visualizações:
[**Documentação YellowBrick**](https://www.scikit-yb.org/en/latest/)



In [ ]:
from yellowbrick.regressor import PredictionError

fig, ax = plt.subplots(figsize=(10, 10))
pev = PredictionError(dtr)
pev.fit(X_treino, y_treino)
pev.score(X_teste, y_teste)
pev.poof()

## 3.3 - Métrica

In [ ]:
#coeficiente de determinacao
r2_dtr = r2_score(y_teste, previsao_dtr)
r2_dtr

# Aula 4 - Random Forest para Regressão


## 4.1 - Modelo Ensemble

[slides]

[Biblioteca Sklearn - Random Forest Regressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor)

**Dica de leitura**: [Artigo Científico - Random Forests](https://link.springer.com/article/10.1023/A:1010933404324), [Wikipédia: Random Forests](https://en.wikipedia.org/wiki/Random_forest#:~:text=Random%20forests%20or%20random%20decision,class%20selected%20by%20most%20trees.), [Capítulo 4 - Livro Introduction to Statistical and Machine Learning Methods for Data Science](https://www.amazon.com.br/dp/B09C2DVZHR/ref=dp-kindle-redirect?_encoding=UTF8&btkr=1), [Capítulo 7 - Mãos à obra: aprendizado de máquina com Scikit-Learn, Keras & TensorFlow: Conceitos, ferramentas e técnicas para a construção de sistemas inteligentes](https://www.amazon.com.br/M%C3%A3os-obra-aprendizado-Scikit-Learn-inteligentes/dp/8550815489/ref=sr_1_2?keywords=machine+learning&qid=1658181249&sprefix=machine%2Caps%2C469&sr=8-2&ufe=app_do%3Aamzn1.fos.6d798eae-cadf-45de-946a-f477d47705b9)

## 4.2 - Previsão

**Ponto de Atenção**
Os parâmetros são praticamente os mesmos entre o `DecisionTreeRegressor()` e o `RandomForestRegressor()`, com exceção de:

* `n_estimators`: quantidade de árvores na floresta
* `bootstrap`: define se será utilizado o método estatístico de amostragem *bootstrap*
* `oob_score`: define se será utilizado o método de amostragem *out-of-bag* para mensurar os erros da predição
* `n_jobs`: número de *jobs* rodando paralelamente (computação paralela - processadores)
* `warm_start`: faz a adequação de uma nova floresta ou usa uma floresta existente.
* `max_samples`: para o caso do `bootstrap` ser `True`, define o número de amostras a serem extraídas de X para treinar cada árvore base.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor(random_state=42, max_depth=10, n_estimators=50)

In [ ]:
rf.fit(X_treino, y_treino)

In [ ]:
previsao_rf = rf.predict(X_teste)

**Gráfico de erro de predição**

In [ ]:
from yellowbrick.regressor import PredictionError

fig, ax = plt.subplots(figsize=(10, 10))
pev = PredictionError(rf)
pev.fit(X_treino, y_treino)
pev.score(X_teste, y_teste)
pev.poof()

## 4.3 - Métrica de desempenho

In [ ]:
#coeficiente de determinacao
r2_rf = r2_score(y_teste, previsao_rf)
r2_rf

# Aula 5 - Técnicas de Otimização


## 5.1 - Ferramentas de Otimização

* Por que otimizar?

* Quais ferramentas temos?

[Melhorando os hiperparâmetros - Documentação Sklearn](https://scikit-learn.org/stable/modules/grid_search.html#tuning-the-hyper-parameters-of-an-estimator)

### 1. Cross Validation (Validação Cruzada)

* [Documentação Cross-validation](https://scikit-learn.org/stable/modules/cross_validation.html)

* **Cross Validation** é uma técnica muito utilizada para avaliação de desempenho de modelos e consequente verificação de tendências no aprendizado de máquina. Ele consiste em particionar os dados em subconjuntos (partes), onde um subconjunto é utilizado para treino e outro subconjunto é utilizado para teste e avaliação do desempenho do modelo.

* Os dados particionados são os de treino (70% dos dados originais)!

* **K-fold**: conjunto de treino é dividido de forma aleatória em K subconjuntos de aproximadamente mesmo tamanho, depois são realizados K iterações, onde K-1 subconjuntos são para treino e 1 para validação.

* O desempenho calculado pela validação cruzada k-fold é a média de todos os valores calculados em cada iteração.


### 2. Randomized Parameter Optimization (Otimização Aleatória de Parâmetros)

* [Documentação Randomized Search](https://scikit-learn.org/stable/modules/grid_search.html#randomized-parameter-optimization)

* [Artigo Científico sobre Random Search](https://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf)

* "Define um espaço de busca como um domínio limitado de valores de hiperparâmetros e pontos de amostra aleatórios nesse domínio."

* Imagem no artigo sobre funcionamento desta busca



## 5.2 - Otimizando a árvore de decisão

In [ ]:
#divisão do dataset em 3 pedaços
from sklearn.model_selection import KFold

#definindo o K - número de subconjuntos
cv = KFold(n_splits = 5, shuffle=True)

### Cross validation


In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
desempenho_cross_val_r2 = cross_val_score(estimator = dtr,
                                       X = X_treino,
                                       y = y_treino,
                                       scoring = 'r2',
                                       cv = cv)
desempenho_cross_val_r2

In [ ]:
print('Desempenho médio de r2: {}'.format(round(desempenho_cross_val_r2.mean(), 3)))

### Random Search Validação

In [ ]:
#seleção de hiperparâmetros
param_random_dt = {
    'criterion': ['squared_error', 'absolute_error'],
    'max_depth': [6, 8, 10, 12, 14, 16],
    'min_samples_leaf': [6, 8, 10],
    'min_samples_split': [8, 10, 12, 14, 16],
    'max_features': ['sqrt', 'log2', 'auto'],
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
random_cv_dt = RandomizedSearchCV(
    dtr,
    param_random_dt,
    n_iter=5,
    cv=cv,
    scoring='r2',
    n_jobs=-1
)

In [ ]:
randomcv_dtr = random_cv_dt.fit(X_treino, y_treino)

In [ ]:
pd.DataFrame(randomcv_dtr.cv_results_)

In [ ]:
#a melhor combinação
randomcv_dtr.best_params_

In [ ]:
#o melhor score
randomcv_dtr.best_score_

### Random Search Teste

In [ ]:
dtr_otimizado = DecisionTreeRegressor(criterion='squared_error' , max_depth=16, max_features='log2' ,
                            min_samples_leaf= 8, min_samples_split =10 )

In [ ]:
dtr_otimizado.fit(X_treino, y_treino)

In [ ]:
predicao_dtr_otimizado = dtr_otimizado.predict(X_teste)

In [ ]:
r2_score(y_teste, predicao_dtr_otimizado)

## 5.3 - Otimizando a Random Forest

### Cross validation

In [ ]:
desempenhoRF_cross_val_r2 = cross_val_score(estimator = rf,
                                       X = X_treino,
                                       y = y_treino,
                                       scoring = 'r2',
                                       cv = cv)
desempenhoRF_cross_val_r2

In [ ]:
print('Desempenho médio de r2: {}'.format(round(desempenhoRF_cross_val_r2.mean(), 3)))

### Random Search Validação

Definindo os parâmetros

In [ ]:
param_random_rf = {
    'n_estimators': [150, 200],
    'max_depth': [20, 30],
    'min_samples_split': [8, 10],
    'min_samples_leaf': [3],
}

In [ ]:
random_cv_rf = RandomizedSearchCV(
    rf,
    param_random_rf,
    n_iter=5,
    cv=cv,
    scoring='r2',
    n_jobs=-1,
    random_state = 42
)

In [ ]:
rf_randomcv = random_cv_rf.fit(X_treino, y_treino)

In [ ]:
pd.DataFrame(rf_randomcv.cv_results_)

In [ ]:
rf_randomcv.best_params_

In [ ]:
rf_randomcv.best_score_

### Random Search Teste

In [ ]:
rf_otimizado2 = RandomForestRegressor(random_state=42, max_depth=30, n_estimators=200,
                                      min_samples_leaf=3, min_samples_split=8)

In [ ]:
rf_otimizado2.fit(X_treino, y_treino)

In [ ]:
predicao_rf_otimizado2 = rf_otimizado2.predict(X_teste)

In [ ]:
r2_score(y_teste, predicao_rf_otimizado2)

## 5.4 - Escolhendo o melhor modelo

In [ ]:
#tabela de resultados
print('Random Search')
print("="*30)
print("Árvore de Decisão: %f" % r2_score(y_teste, predicao_dtr_otimizado))
print("Floresta Aleatória: %f" % r2_score(y_teste, predicao_rf_otimizado2))
print("*"*30)

print("Cross Validation")
print("="*30)
print("Árvore de Decisão: %f" % round(desempenho_cross_val_r2.mean(), 3))
print("Floresta Aleatória: %f" % round(desempenhoRF_cross_val_r2.mean(), 3))

In [ ]:
#Testando o melhor modelo
cliente_joana = pd.DataFrame({'tax_iptu':[4.6], 'tax_condo':[5.45], 'bedrooms':[2],	'bathrooms':[1], 'suites':[1], 'parkingSpaces':[1],	'usableAreas':[4],	'totalAreas':[4.68],	'floors':[0],	'unitsOnTheFloor':[0],	'unitFloor':[0],	'unit_Apartamento':[0],	'unit_Casa':[1],	'unit_Outros':[0],	'zone_Zona Central':[0],	'zone_Zona Norte':[0],	'zone_Zona Oeste':[1],	'zone_Zona Sul':[0]})
cliente_joana

In [ ]:
#Verificando resultados
predicao_cliente_joana = rf_otimizado2.predict(cliente_joana)
np.expm1(predicao_cliente_joana)